In [1]:
%%writefile mapper.py

import sys
import re
import string

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stop_words = ["a", "able", "about", "across", "after", "all", "almost", "also", "am", "among", "an", "and", "any", "are", "as", "at", "be", "because", "been", "but", "by", "can", "cannot", "could", "dear", "did", "do", "does", "either", "else", "ever", "every", "for", "from", "get", "got", "had", "has", "have", "he", "her", "hers", "him", "his", "how", "however", "i", "if", "in", "into", "is", "it", "its", "just", "least", "let", "like", "likely", "may", "me", "might", "most", "must", "my", "neither", "no", "nor", "not", "of", "off", "often", "on", "only", "or", "other", "our", "own", "rather", "said", "say", "says", "she", "should", "since", "so", "some", "than", "that", "the", "their", "them", "then", "there", "these", "they", "this", "tis", "to", "too", "twas", "us", "wants", "was", "we", "were", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "would", "yet", "you", "your"]
numbers = ["0","1","2","3","4","5","6","7","8","9"]

def clean_word(word):
    for c in string.punctuation:
        word = word.replace(c,"")
    for c in numbers:
        word = word.replace(c,"")
    return word

def dont_stop(word):
    if word in stop_words or word == "":
        return False
    else:
        return True
    


for line in sys.stdin:
    l = line.strip().split()
    for word in l:
        word = clean_word(word)
        if dont_stop(word) and len(word) == 7:
            print "%s\t%d" %(word, 1)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys
import os

word_list = {}

Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

def inv_word_list(word_list):
    inv_word_list = []
    for key, value in word_list.item():
        inv_word_list.append((value, key))
    return sorted((inv_word_list, reverse == True))


for line in sys.stdin:
    word, count = line.strip().split("\t", 1)
    
    try:
        count = int(count)
    except ValueError:
        continue
        
    if word not in word_list:
        word_list[word] = count
    else:
        word_list[word] += count
        
inv_word_list = inv_word_list(word_list)

for word in inv_word_list[:10]:
    print "%s\t%s" %(word[1], word[0])

Appending to reducer.py


In [ ]:
%%bash

##locate hadoop-streaming jar using the following code

find / -name 'hadoop-streaming*.jar'

In [ ]:
%%bash
hdfs dfs -ls -h /data/wiki

In [4]:
%%bash

OUT_DIR="Top-K_wordsCount_result_"$(date +"%s%6N")
NUM_REDUCERS=8   
# if the number of reducer is 0 then no sort and suffle phase will be invoked and only mapper will work. If its 1 
#then sort and suffle will be invoked and 1 output file will be generated i.e. only 1 reducer. If its 8 then 
#after sorf and suffle there will be 8 reducers. In all case word count will be same.

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming TopWordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

rm: `Top-K_wordsCount_result_1568455949486323': No such file or directory
19/09/14 10:12:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/14 10:12:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/14 10:12:43 INFO mapred.FileInputFormat: Total input files to process : 1
19/09/14 10:12:43 INFO mapreduce.JobSubmitter: number of splits:2
19/09/14 10:12:44 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568453614625_0002
19/09/14 10:12:45 INFO impl.YarnClientImpl: Submitted application application_1568453614625_0002
19/09/14 10:12:45 INFO mapreduce.Job: The url to track the job: http://c8475de0a7e5:8088/proxy/application_1568453614625_0002/
19/09/14 10:12:45 INFO mapreduce.Job: Running job: job_1568453614625_0002
19/09/14 10:13:00 INFO mapreduce.Job: Job job_1568453614625_0002 running in uber mode : false
19/09/14 10:13:00 INFO mapreduce.Job:  map 0% reduce 0%
19/09/14 10:13:27 INFO mapreduce.Job:  map 3% reduce 0%
19/09/1

In [ ]:
%%bash

#hdfs dfs -ls -h  /user/jovyan

#hdfs dfs -ls -h /user/jovyan/wordcount_result_1568442801044255

hdfs dfs -text /user/jovyan/wordcount_result_1568442801044255/part-00001 | tail
